# Pre-Requisite 

## Different Trader Teams
    - epsilon_trader
    - opt_trader
    - granite_trader
    - carle_trader

## checking the no of cores on servers
    - lscpu (command)
    - Thread(s) per core:  1
    - NUMA node0 CPU(s):   0-15

### CPU are isolated or not 
- To check whether cpu are isolated or not , we can run below command and and check isolcpus
- command- **cat /proc/cmdline/**

- for eg:<br>
[pratima@nemo17 ~]$ cat /proc/cmdline <br>
 BOOT_IMAGE=(hd0,gpt2)/vmlinuz-4.18.0-80.el8.x86_64 root=/dev/mapper/rhel-root ro crashkernel=auto resume=/dev/mapper/rhel-swap rd.lvm.lv=rhel/root rd.lvm.lv=rhel/swap rhgb quiet net.ifnames=0 biosdevname=0 **isolcpus=1-19**

###  status of sfptpd 
- To verify wether sfptpd is enable or not. we run below command and check if it is 'Active ' or not
- command **systemctl status sfptpd**
- for eg:
[pratima@nemo17 ~]$ systemctl status sfptpd <br>
● sfptpd.service - sfptpd <br>
   Loaded: loaded (/usr/lib/systemd/system/sfptpd.service; enabled; vendor preset: disabled)<br>
   Active: **active (running)** since Fri 2021-06-04 12:10:46 IST; 1 weeks 4 days ago<br>
 Main PID: 3671 (sfptpd)<br>
    Tasks: 4 (limit: 26213)<br>
   Memory: 108.7M<br>
   CGroup: /system.slice/sfptpd.service<br>
           └─3671 /usr/sbin/sfptpd -f/etc/sfptpd.conf<br>

### Synchronization of time 

- It coulp be possible that system is running but synchrinizing the time.
- To check this we run cat /var/lib/sfptpd/topology/

- [pratima@nemo17 ~]$ cat /var/lib/sfptpd/topology <br>
   \====================<br>
    state: ptp-slave <br>
    interface: fpga-in (fpga-in)<br>
    timestamping: hw<br>
   \====================

            grandmaster            
        000e:feff:fe01:01bc        
                 |                 
                 |                 
              5 steps              
                 |                 
                 |                 
                 v                 
              parent               
      00ea:bdff:fe64:90fc/11       
                 |                 
                 |                 
             -7.000 ns             
                 |                 
                 |                 
                 v                 
        phc3(sfc0/fpga-in)         
        000f:53ff:fe61:ef90        
                 |                 
                 |                 
                 o----------------------------------o                 
                 |                                  |                 
                 |                                  |                 
             -2.062 ns                         -14.750 ns             
                 |                                  |                 
                 |                                  |                 
                 v                                  v                 
     phc2(L3feed/interactive)                    system               
        000f:53ff:fe68:e3a0                      system 

# Steps to Set Up  Epsilon Trader Server

###    Login to trader team user account for which we are doing set up . Configuring Tfeed Process
    eg:
    epsilon_trader
 * **command :** <br>
     --> sudo -u epsilon_trader -i
 * **creating Required directories and symlinks**  <br>
    mkdir nse_tbt** <br>
    mkdir /mnt/huge/epsilon_trader <br>
    mkdir /data/epsilon_trader_keys <br>
    mkdir infra_scripts <br>
    mkdir nse_tbt <br>
      **creating all_logs directory and symlink log in nse_tbt directory**<br>
      cd nse_tbt <br>
      mkdir all_logs <br>
      ln -s all_logs log <br>

 * **creating symlinks**<br>
     ln -s /mnt/huge/epsilon_trader/ live_logs <br>
     ln -s /data/epsilon_trader_keys keys
      
     
   

### need to create required config:
- tfeed.cfg
- tokens.cfg

### Need to touch two files in /data/epsilon_trader_keys directory 
- touch tfeed_cb
- touch tfeed_token_status

### content on tfeed.cfg and paramters to verify once 
free_list_size=20000 <br>
num_segments=4 <br>
**keys_dir=/data/epsilon_trader_keys/**<br>
cd_contract_path=/data/contracts/cd_contract.txt<br>
fo_contract_path=/data/contracts/contract.txt<br>
fo_spd_contract_path=/data/contracts/fo_spd_contract.txt<br>
cd_spd_contract_path=/data/contracts/cd_spd_contract.txt<br>
cm_security_path=/data/contracts/security.txt<br>
cd_masters_data_path=/data/contracts/cd_contract_stream_info.csv<br>
fo_masters_data_path=/data/contracts/fo_contract_stream_info.csv<br>
cm_masters_data_path=/data/contracts/cm_contract_stream_info.csv<br>
cd_spread_masters_data_path=/data/contracts/cd_spd_contract_stream_info.csv<br>
fo_spread_masters_data_path=/data/contracts/fo_spd_contract_stream_info.csv<br>
cm_spread_masters_data_path=/data/contracts/cm_spd_contract_stream_info.csv<br>
nse_banned_securities=/data/contracts/NSE_banned_securities.csv<br>
fo_secban=/data/contracts/fo_secban.csv<br>
base_shm_key=34897<br>
interface_ip=0.0.0.0<br>
#network_interface=L3feed<br>
**network_interface=sfc0**<br>
max_streams=2<br>
max_listen_channels=1<br>
socket_recvbuf=16777216<br>
**num_reader_threads=2**<br>
**cpu_ids_for_multicast_threads=1**<br>
**cpu_ids_for_multicast_threads=2**<br>
#cpu_ids_for_multicast_threads=3<br>
#cpu_ids_for_multicast_threads=4<br>
**cpu_ids_for_recovery_threads=0**<br>
**background_threads_cpu_ids=0**<br>
hold_crossing_mbos=false<br>
**#multicast_provider_type=tcp_direct**<br>
**multicast_provider_type=ef_vi**<br>
**mdi_key_prefix=/epsilon_trader_nse_mdi**<br>


<font color='green'>
Note:This is raw set up without any specific details on server.Need to get this like scripts/config from a specific host which is up-to-date
 </font>

# Set up for TS on server 

### take login to TS :
- **sudo -u tradesrv1 -i**

- **creating Required directories and symlinks**  <br>
    mkdir -p /mnt/huge/tradesrv1 <br>
    mkdir rms_server <br>
    mkdir nse_trade_server <br>
    mkdir -p nse_trade_server/scripts <br>

- **creating symlinks**<br>
     ln -s /mnt/huge/tradesrv1 live_logs <br>
     
- **creating all_logs directory and symlink log in <font color='Red'>nse_trade_server</font> directory**<br>
      cd nse_trade_server <br>
      mkdir all_logs <br>
      ln -s all_logs log<br>
      ln -s /data/nse_trade_server nse_native_trade_server <br>
      ln -s /usr/share/infra_configs/tradesrv1/_config_template.cfg  _config_template.cfg <br>


- **creating all_logs directory and symlink log in <font color='Red'>rms_server</font> directory**<br>
      cd rms_server <br>
      mkdir all_logs <br>
      ln -s all_logs log<br>
      ln -s /data/rms_server rms_server <br>
          
      
     

### creating <font color='Red'> \_config_templare.cfg </font>config. Parametrs needs to verify
- vi \_config_template.cfg

local_listen_ip=127.0.0.1<br>
**server_port_number=25100**<br>
shm_per_strat_size=1048576<br>
cancel_orders_flag=false<br>
max_tokens_per_strategy=128<br>
rms_server_ip=127.0.0.1<br>
**rms_server_port=8100**<br>
#security_repository_shm_key_dir=/data/epsilon_trader_keys/<br>
**info_module_config_file=/data/epsilon_shm_info_module.cfg**<br>
#UserID|Password|NewPassword|BrokerID|BranchID|NSE-version-number|UserType|ComplianceID|IP-Address|Port|Segment|Local-IP<br>
**connection=37668|__CURRENT_PASS_1__|__NEW_PASS_1__|13871|3|27734|24860|74400|0|201012002009000|127.0.0.1|10825|FO|10.230.40.133|95|AAICA0383M**<br>
cpu_ids_for_tradeserver=0<br>
**poll_for_orders_cpu_id=6**<br>
enable_shm_process_mode=true<br>
enable_network_mode=false<br>
prefetch_security_info=false<br>
epoll_timeout=0<br>
**raw_mode_exchange_ip=172.19.13.85**<br>
**#tcp_type=tcp_direct**<br>
**#exchange_interface=L1feedB**<br>
#exit_on_exchange_disconnect=true<br>

### Depends on the number of connection present in _config_template .cfg  need to create directory password_management_{connection_number} --> 1,2,3...etc


- cd nse_trade_server
    - mkdir password_management_1
    
content of pasword_mangement_1 <br>
- next_pass.txt
- current_pass.txt
- all_passwords.txt
- change_date.txt
- auto_start.log

<br>
<font color='Red' <br>
      Note : for the first time session to be use , need to update "all_passowrd.txt" and keep default password which depend on segment  </font>

FO - Neat@FO1<br>
CD - Neat@CD1<br>
CM - Neat@CM1<br>

vi all_password.txt <br>
**Neat@FO1**<br>
Cshk@SD8<br>
Fgas@UH6<br>
Bhfg@DF4<br>
Pkgy@BI6<br>
Fofg@SD5<br>
Edpo@FG2<br>
Rtsp@DF1<br>
Jigh@SD4<br>
Phgf@CF3<br>
Amrt@RG8<br>
Tues@NY1<br>
Fvgh@VC4<br>
Bnkh@CG3<br>

- After saving the file. Need to touch "force_change.txt" file . This is done when we start TS , the script will automatically forcefuly change to the deafult password.  
<br>
<font color='Red'> <br>
Note :Once the password is set, then NeatFO1 should be removed from all_passwords.txt. Similarly other default password<br>
</font>